<a href="https://colab.research.google.com/github/parantou/OLLA/blob/Colab/%EA%B0%90%EC%84%B1%EC%A7%80%EC%88%98_%EC%B6%9C%EB%A0%A5_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/kakaobrain/pororo.git

Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 22.20 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [ ]:
cd pororo

/content/pororo


In [ ]:
!pip3 install torch==1.13.0
!pip3 install torch>=1.7
!pip3 install torchvision>=0.8.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████▋                   | 351.9 MB 11.0 MB/s eta 0:00:50

In [ ]:
!pip install --use-deprecated=legacy-resolver

In [ ]:
import pandas as pd

test_df = pd.read_csv("/content/drive/MyDrive/SK2-22.csv") # LG 뉴스.csv 불러오기
test_df.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,stock,title,company,date,content
0,2781,2912,2912,sk하이닉스,"[특징주] 삼성전자·SK하이닉스, 동반 52주 신저가 추락(종합)",연합뉴스,2022-09-22,삼성전자 SK하이닉스 (CG)[연합뉴스TV 제공] (서울=연합뉴스) 김아람 기...
1,2782,2913,2913,sk하이닉스,삼성전자-SK하이닉스 동맹? ARM '적과의 동침' 가능성은,뉴시스,2022-09-22,"기사내용 요약삼성전자, SK하이닉스·퀄컴·인텔 등 관계사와 컨소시엄 가능성[서울=뉴..."
2,2783,2914,2914,sk하이닉스,"'FOMC 충격 여파' 삼성전자·SK하이닉스, 반도체 대장株 신저가 추락",머니S,2022-09-22,미국 연방준비제도(Fed·연준)의 3연속 자이언트스텝 단행 여파로 삼성전자가 52주...
3,2784,2915,2915,sk하이닉스,"'4만전자' 코앞, SK하이닉스는 '86층'…반도체株도 동반 신저가",머니투데이,2022-09-22,/사진=임종철 디자인기자코스피를 떠받들던 국내 반도체주가 속절없이 무너지면서 삼성전...
4,2785,2916,2916,sk하이닉스,"[특징주]삼성전자·SK하이닉스, 장 초반 52주 신저가 나란히 경신",아시아경제,2022-09-22,삼성전자와 SK하이닉스가 22일 장 초반 52주 신저가 기록을 갈아치웠다. 전날 미...


In [ ]:
from pororo import Pororo
import numpy as np

total = np.array(test_df['title'].str.cat(test_df['content'])) # 뉴스 제목과 본문 합친다.

text_sum = Pororo(task="text_summarization", lang="ko", model="abstractive")  # 본문 요약 (Pororo)

# 요약 내용을 담을 리스트
summary_list = []

# Pororo 포문으로 돌려서 요약한 내용
for i in total:
  summary_list.append(text_sum(i))

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

IndexError: ignored

In [ ]:
'''
감성분석에 hugginface에서 제공하는 사전학습 모델인 KR-FinBERT-SC 모델을 사용
(https://huggingface.co/snunlp/KR-FinBert-SC) KR-FinBERT-SC는 FinBERT 모델을 기반으로 한국어 경제 기사를 학습한 모델이다.
감성분석을 위해 학습되었으며 긍정, 부정, 중립 수치가 출력된다.
https://dacon.io/competitions/official/235914/codeshare/5550
'''
from argparse import Namespace
from sklearn.metrics import classification_report
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# 사전학습 모델 로드
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
sa = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer) # 감성분류 모델 객체 sa 생성

totalsa_list = [] # 뉴스(제목+내용) sa(sentiment analysis) 배열 초기화
for news in summary_list:
    inputs = tokenizer(news, return_tensors='pt')
    output = model(**inputs)
    output = output.logits.tolist()[0]
    totalsa_list.append(output)

In [ ]:
import torch
import torch.nn as nn

outputs = torch.tensor(totalsa_list) # 출력값을 텐서로 변환 # outputs

predictions = nn.functional.softmax(outputs, dim=-1) # 출력값을 확률 형태로 변환 # predictions

In [ ]:
# 출력값 확인
df_sd = pd.DataFrame(predictions.detach().numpy())
df_sd.columns = ['negative','neutral','positive']
df_sd

,negative,neutral,positive
0,0.153839,0.096778,0.749383
1,0.028643,0.023599,0.947758
2,0.723889,0.028944,0.247167
3,0.666395,0.292137,0.041468
4,0.133914,0.005486,0.860600
...,...,...,...
1217,0.000037,0.000084,0.999879
1218,0.001733,0.000225,0.998041
1219,0.000039,0.000080,0.999881
1220,0.000044,0.000070,0.999886


In [ ]:
lgsa_df = pd.concat([test_df[['date','stock']], df_sd], axis=1) # 날짜와 종목명 감성수치 합치기
#lgsa_df = pd.concat([test_df, df_sd], axis=1) # 날짜와 감성수치 합치기

lgsa_df.head()

,date,negative,neutral,positive
0,2022-01-09,0.153839,0.096778,0.749383
1,2022-01-09,0.028643,0.023599,0.947758
2,2022-01-09,0.723889,0.028944,0.247167
3,2022-01-09,0.666395,0.292137,0.041468
4,2022-01-09,0.133914,0.005486,0.860600


In [ ]:
# 일자 별 감성수치 평균
df_group = lgsa_df.groupby(['date','stock']).mean()
df_group.head()

,negative,neutral,positive
date,,,
2021-12-09,0.000796,0.352522,0.646682
2021-12-10,0.005360,0.255872,0.738768
2021-12-14,0.000797,0.225049,0.774153
2021-12-15,0.000143,0.000604,0.999253
2021-12-18,0.007270,0.001017,0.991713


In [ ]:
df_group['logit'] = np.log(df_group['positive']/df_group['negative']) # logit = ln(P/N)
df_group.head()

,negative,neutral,positive,logit
date,,,,
2021-12-09,0.000796,0.352522,0.646682,6.700464
2021-12-10,0.005360,0.255872,0.738768,4.925962
2021-12-14,0.000797,0.225049,0.774153,6.878093
2021-12-15,0.000143,0.000604,0.999253,8.849360
2021-12-18,0.007270,0.001017,0.991713,4.915714


In [ ]:
df_group['intensity'] = np.log(1 + (test_df.groupby(['date','stock'])['content'].count()/test_df['content'].count())) # intensity = ln(1 + n(해당 일자에 작성된 뉴스수)/(avg of N))
# sentiment_index = logit*intensity

df_group

,negative,neutral,positive,logit,intensity
date,,,,,
2021-12-09,0.000796,0.352522,0.646682,6.700464,0.002452
2021-12-10,0.005360,0.255872,0.738768,4.925962,0.000818
2021-12-14,0.000797,0.225049,0.774153,6.878093,0.009772
2021-12-15,0.000143,0.000604,0.999253,8.849360,0.002452
2021-12-18,0.007270,0.001017,0.991713,4.915714,0.000818
...,...,...,...,...,...
2022-11-22,0.009743,0.901962,0.088294,2.204100,0.001635
2022-11-24,0.001433,0.915533,0.083035,4.059702,0.011391
2022-11-30,0.001865,0.002289,0.995846,6.280109,0.001635


In [ ]:
df_group['sent_index'] = df_group['logit'] * df_group['intensity'] # 감성지표 완성
df_group

,negative,positive,logit,intensity,sent_index
date,,,,,
2021-12-09,0.000796,0.646682,6.700464,0.002452,0.016429
2021-12-10,0.005360,0.738768,4.925962,0.000818,0.004029
2021-12-14,0.000797,0.774153,6.878093,0.009772,0.067213
2021-12-15,0.000143,0.999253,8.849360,0.002452,0.021698
2021-12-18,0.007270,0.991713,4.915714,0.000818,0.004021
...,...,...,...,...,...
2022-11-22,0.009743,0.088294,2.204100,0.001635,0.003604
2022-11-24,0.001433,0.083035,4.059702,0.011391,0.046246
2022-11-30,0.001865,0.995846,6.280109,0.001635,0.010270


In [ ]:
df_group.to_csv('/content/drive/MyDrive/LGSI_dataset')